<div style="background-color:Brown;"><h1 style="color:Orange;text-align:center; font-size:40px;">  Web Scraping - Mission to Mars</h1> </div>

### Importing dependencies

In [1]:
import sys
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver  # using selenium to obtain the whole webpage content due to JS implementation in the website
!{sys.executable} -m pip install chromedriver_autoinstaller
import chromedriver_autoinstaller # needed for the proper operation of Selenium

In [2]:
chromedriver_autoinstaller.install() # Automatically finds the correct Driver based on the currently installed chromes version 
executable_path = {'executable_path': ChromeDriverManager().install()} # Executable path needed for splinter's correct operation

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
INFO:WDM:Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
INFO:WDM:Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


<h1 style="color:blue;text-align:center; font-size:40px;"> Scraping </h1>

<h2 style="color:green;text-align:center;"> NASA Mars News </h2>

In [3]:
# Mars News site
url = "https://redplanetscience.com/"

In [4]:
# instantiating the webdriver for Chrome!!!
opt = webdriver.ChromeOptions()
opt.add_argument("--start-minimized")
driver = webdriver.Chrome(options=opt)
# Getting the webpage content
driver.get(url)
# parsing our html plain text to a BS object
soup = BeautifulSoup(driver.page_source, 'html.parser')
# Closing the driver(chrome instance)
driver.quit()
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [5]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')[0]  # using index 0 to get only the lastest news Title
# Storing the news title
news_title = results.find('div',class_='content_title').text
news_p = results.find('div',class_='article_teaser_body').text

In [6]:
print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

Title: NASA Invites Students to Name Mars 2020 Rover
Paragraph: Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


<h2 style="color:green;text-align:center;"> JPL Mars Space Images - Featured Image </h2>

In [7]:
# instantiating our browser object
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://spaceimages-mars.com/'
# open browser and go to https://spaceimages-mars.com/
browser.visit(url)

In [8]:
# obtaining featured image, knowing beforehand that the featured image tag has a class named "headerimage fade-in"
featured_image_url = browser.find_by_css('img[class="headerimage fade-in"]')['src'] # getting the src Attribute(image url path)
#closing the browser
browser.quit()

In [9]:
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

<h2 style="color:green;text-align:center;"> Mars Facts </h2>

In [10]:
url = 'https://galaxyfacts-mars.com/'

In [11]:
# Use Panda's `read_html` to parse the url
tables = pd.read_html(url)

In [12]:
len(tables)

2

In [13]:
# Finding the table to use in our flask app
tables[0]

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df = tables[0]

##### Cleaning up table

In [15]:
df.columns = ['Description','Mars','Earth']
df.head()

,Description,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [16]:
# Resetting index
df.set_index(df.Description,inplace=True)
df

,Description,Mars,Earth
Description,,,
Mars - Earth Comparison,Mars - Earth Comparison,Mars,Earth
Diameter:,Diameter:,"6,779 km","12,742 km"
Mass:,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,Moons:,2,1
Distance from Sun:,Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,Length of Year:,687 Earth days,365.24 days
Temperature:,Temperature:,-87 to -5 °C,-88 to 58°C


In [17]:
html_table = df.to_html()

In [18]:
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>Length of Year:</td>
      <td>687 

<h2 style="color:green;text-align:center;">Mars Hemispheres </h2>

In [19]:
#list that will be used to store titles and links to high resolution images
hemisphere_image_urls = []

In [20]:
# instantiating our browser object
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://marshemispheres.com/'
# open browser and go to https://marshemispheres.com/
browser.visit(url)

In [21]:
# creating a splinter.element_list.ElementList
hemispheres = browser.links.find_by_partial_text('Enhanced')

In [22]:
for i in range(0, len(hemispheres)):
    browser.links.find_by_partial_text('Enhanced')[i].click()
    title = browser.find_by_css('h2[class="title"]').text
    img_url = browser.links.find_by_partial_text('Original')['href']
    hemisphere_image_urls.append({"title": title, "img_url": img_url})
    browser.visit(url)
browser.visit("https://google.com")

In [23]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]


In [24]:
browser.quit()